In [1]:
import numpy 
import pandas 
import tensorflow 
import nltk
import gensim

print(numpy.__version__)
print(pandas.__version__)
print(tensorflow.__version__)
print(nltk.__version__)
print(gensim.__version__)


1.21.4
1.3.3
2.6.0
3.6.5
3.8.3


In [2]:
import os
import pandas as pd

data_path = os.getenv('HOME')+'/aiffel/songys_chatbot'
chat = pd.read_csv(data_path+'/ChatbotData.csv')
chat.head()

,Q,A,label
0,12시 땡!,하루가 또 가네요.,0
1,1지망 학교 떨어졌어,위로해 드립니다.,0
2,3박4일 놀러가고 싶다,여행은 언제나 좋죠.,0
3,3박4일 정도 놀러가고 싶다,여행은 언제나 좋죠.,0
4,PPL 심하네,눈살이 찌푸려지죠.,0


In [3]:
questions = chat['Q'].values.tolist()
answers = chat['A'].values.tolist() 

아래 조건을 만족하는 preprocess_sentence() 함수를 구현하세요.

- 영문자의 경우, 모두 소문자로 변환합니다.
- 영문자와 한글, 숫자, 그리고 주요 특수문자를 제외하곤 정규식을 활용하여 모두 제거합니다.
- 문장부호 양옆에 공백을 추가하는 등 이전과 다르게 생략된 기능들은 우리가 사용할 토크나이저가 지원하기 때문에 굳이 구현하지 않아도 괜찮습니다!

In [4]:
import re

def preprocess_sentence(sentence):
    new_sen = []    
    for i in sentence:
        i = i.lower()
        i = re.sub(r"[^ㄱ-ㅎㅏ-ㅣ가-힣 a-z A-Z?.!,0-9\\s]+", "", i)
        new_sen.append(i)
    return new_sen


In [5]:
from konlpy.tag import Mecab

def build_corpus(src, tgt):
    tokenizer = Mecab()
     
    src_ = preprocess_sentence(src)
    tgt_ = preprocess_sentence(tgt)
    
    que_corpus = []
    ans_corpus = []
    
    for i in src_:
        que_corpus.append(tokenizer.morphs(i))
    
    for i in tgt_: 
        ans_corpus.append(tokenizer.morphs(i))
    
    return que_corpus, ans_corpus

In [6]:
que_corpus, ans_corpus = build_corpus(questions, answers)

print(len(que_corpus))
print(len(ans_corpus))

11823
11823


In [7]:
print(que_corpus[0])
print(ans_corpus[0])

['12', '시', '땡', '!']
['하루', '가', '또', '가', '네요', '.']


In [7]:
# !pip install --upgrade gensim==3.8.3
# # !pip uninstall gensim
# # !pip install gensim==3.8.3

In [8]:
print(gensim.__version__)

3.8.3


In [9]:
import gensim

wv = gensim.models.Word2Vec.load(data_path+'/ko/ko.bin')

In [10]:
# Lexical Substitution 구현하기
import random

def lexical_sub(sentence, word2vec):

    res = []
    toks = sentence

    try:
        _from = random.choice(toks)
        _to = word2vec.most_similar(_from)[0][0]

    except:   # 단어장에 없는 단어
        return None

    for tok in toks:
        if tok is _from: res.append(_to)
        else: res.append(tok)

    return res

In [57]:
#이전 결과 기록용
arg_que_corpus = []
arg_ans_corpus = []

for j in range(3):
    if j==0:
        for i in tqdm(range(len(que_corpus))):
            que_temp = lexical_sub(que_corpus[i], wv)
            if que_temp == None: #단어장에 없어서 lexical_sub가 안되는 경우 제외 진행
                pass
            else:
                arg_que_corpus.append(que_temp)
                arg_ans_corpus.append(ans_corpus[i])
    elif j==1:
        for i in tqdm(range(len(que_corpus))):
            ans_temp = lexical_sub(ans_corpus[i], wv)
            if ans_temp == None: #단어장에 없어서 lexical_sub가 안되는 경우 제외 진행
                pass
            else:
                arg_que_corpus.append(que_corpus[i])
                arg_ans_corpus.append(ans_temp)
    elif j==2:
        for i in tqdm(range(len(que_corpus))):
            arg_que_corpus.append(que_corpus[i])
            arg_ans_corpus.append(ans_corpus[i])
    
print(len(arg_que_corpus))
print(len(arg_ans_corpus))

  0%|          | 0/11823 [00:00<?, ?it/s]/tmp/ipykernel_132/269584872.py:11: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  _to = word2vec.most_similar(_from)[0][0]
100%|██████████| 11823/11823 [00:00<00:00, 300017.28it/s]

35469
35469


In [12]:
from tqdm import tqdm
arg_que_corpus = []
arg_ans_corpus = []

for j in range(3):
    if j==0:
        for i in tqdm(range(len(que_corpus))):
            que_temp = lexical_sub(que_corpus[i], wv)
            if que_temp == None: #단어장에 없어서 lexical_sub가 안되는 경우 제외 진행
                pass
            else:
                arg_que_corpus.append(que_temp)
                arg_ans_corpus.append(ans_corpus[i])
    elif j==1:
        for i in tqdm(range(len(que_corpus))):
            ans_temp = lexical_sub(ans_corpus[i], wv)
            if ans_temp == None: #단어장에 없어서 lexical_sub가 안되는 경우 제외 진행
                pass
            else:
                arg_que_corpus.append(que_corpus[i])
                arg_ans_corpus.append(ans_temp)
    elif j==2:
        for i in tqdm(range(len(que_corpus))):
            arg_que_corpus.append(que_corpus[i])
            arg_ans_corpus.append(ans_corpus[i])
    
print(len(arg_que_corpus))
print(len(arg_ans_corpus))

  0%|          | 0/11823 [00:00<?, ?it/s]/tmp/ipykernel_154/269584872.py:11: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  _to = word2vec.most_similar(_from)[0][0]
100%|██████████| 11823/11823 [00:00<00:00, 587989.33it/s]

32291
32291


In [ ]:
#약 3000개 삭제됨

In [72]:
arg_que_corpus[:2]

[['1', '지망', '학교', '떨어졌', '어서'],
 ['3', '박', '4', '일', '놀', '러', '가', '고', '겠', '다']]

In [42]:
# arg_ans_corpus[:-500]

In [15]:
enc_train=[]
dec_train=[]
for i in arg_que_corpus:
    enc_train.append(i)
    
for j in arg_ans_corpus:
    dec_train.append(["<start>"] + j + ["<end>"])


print(len(enc_train))
print(len(dec_train))

32291
32291


In [16]:
enc_train[0]

['1', '지망', '학교', '떨어졌', '어서']

In [17]:
dec_train[0]

['<start>', '위로', '해', '드립니다', '.', '<end>']

In [90]:
from tensorflow.keras.preprocessing.text import Tokenizer
tokenizer = Tokenizer()
tokenizer.fit_on_texts(enc_train)
tokenizer.fit_on_texts(dec_train)

In [22]:
print('mecab word index : '+str(len(tokenizer.word_index)))

mecab word index : 7937


In [24]:
enc_train_ = tokenizer.texts_to_sequences(enc_train)
dec_train_ = tokenizer.texts_to_sequences(dec_train)

In [34]:
enc_train_[:5]
dec_train_[:5]

[[2, 552, 13, 1591, 1, 3],
 [2, 251, 17, 732, 12, 35, 1, 3],
 [2, 251, 17, 732, 12, 35, 1, 3],
 [2, 4509, 4, 4510, 18, 35, 1, 3],
 [2, 49, 120, 10, 15, 7, 41, 21, 15, 30, 1, 3]]

In [38]:
import tensorflow as tf

MAX_LEN = 50
enc_ndarray = tf.keras.preprocessing.sequence.pad_sequences(enc_train_, maxlen=MAX_LEN, padding='post')
dec_ndarray = tf.keras.preprocessing.sequence.pad_sequences(dec_train_, maxlen=MAX_LEN, padding='post')

print('슝=3')

슝=3


In [39]:
enc_ndarray

array([[ 299, 4691,  540, ...,    0,    0,    0],
       [ 332, 2319,  591, ...,    0,    0,    0],
       [ 332, 2319,  591, ...,    0,    0,    0],
       ...,
       [6055,   13,   54, ...,    0,    0,    0],
       [ 217,  108,   12, ...,    0,    0,    0],
       [  69,  119,  160, ...,    0,    0,    0]], dtype=int32)

In [40]:
BATCH_SIZE = 64
train_dataset = tf.data.Dataset.from_tensor_slices((enc_ndarray, dec_ndarray)).batch(batch_size=BATCH_SIZE)

print('슝=3')

슝=3


## transfomer만들기

In [43]:
# Positional Encoding 구현
def positional_encoding(pos, d_model):
    def cal_angle(position, i):
        return position / np.power(10000, (2*(i//2)) / np.float32(d_model))

    def get_posi_angle_vec(position):
        return [cal_angle(position, i) for i in range(d_model)]

    sinusoid_table = np.array([get_posi_angle_vec(pos_i) for pos_i in range(pos)])

    sinusoid_table[:, 0::2] = np.sin(sinusoid_table[:, 0::2])
    sinusoid_table[:, 1::2] = np.cos(sinusoid_table[:, 1::2])

    return sinusoid_table

In [44]:
#마스크 생성
def generate_padding_mask(seq):
    seq = tf.cast(tf.math.equal(seq, 0), tf.float32)
    return seq[:, tf.newaxis, tf.newaxis, :]

def generate_lookahead_mask(size):
    mask = 1 - tf.linalg.band_part(tf.ones((size, size)), -1, 0)
    return mask

def generate_masks(src, tgt):
    enc_mask = generate_padding_mask(src)
    dec_enc_mask = generate_padding_mask(src)

    dec_lookahead_mask = generate_lookahead_mask(tgt.shape[1])
    dec_tgt_padding_mask = generate_padding_mask(tgt)
    dec_mask = tf.maximum(dec_tgt_padding_mask, dec_lookahead_mask)

    return enc_mask, dec_enc_mask, dec_mask
# generate_padding_mask

In [45]:
#multi head attention
class MultiHeadAttention(tf.keras.layers.Layer):
    def __init__(self, d_model, num_heads):
        super(MultiHeadAttention, self).__init__()
        self.num_heads = num_heads
        self.d_model = d_model
        
        self.depth = d_model // self.num_heads
        
        self.W_q = tf.keras.layers.Dense(d_model)
        self.W_k = tf.keras.layers.Dense(d_model)
        self.W_v = tf.keras.layers.Dense(d_model)
        
        self.linear = tf.keras.layers.Dense(d_model)

    def scaled_dot_product_attention(self, Q, K, V, mask):
        d_k = tf.cast(K.shape[-1], tf.float32)
        QK = tf.matmul(Q, K, transpose_b=True)

        scaled_qk = QK / tf.math.sqrt(d_k)

        if mask is not None: scaled_qk += (mask * -1e9)  

        attentions = tf.nn.softmax(scaled_qk, axis=-1)
        out = tf.matmul(attentions, V)

        return out, attentions
        

    def split_heads(self, x):
        bsz = x.shape[0]
        split_x = tf.reshape(x, (bsz, -1, self.num_heads, self.depth))
        split_x = tf.transpose(split_x, perm=[0, 2, 1, 3])

        return split_x

    def combine_heads(self, x):
        bsz = x.shape[0]
        combined_x = tf.transpose(x, perm=[0, 2, 1, 3])
        combined_x = tf.reshape(combined_x, (bsz, -1, self.d_model))

        return combined_x

    
    def call(self, Q, K, V, mask):
        WQ = self.W_q(Q)
        WK = self.W_k(K)
        WV = self.W_v(V)
        
        WQ_splits = self.split_heads(WQ)
        WK_splits = self.split_heads(WK)
        WV_splits = self.split_heads(WV)
        
        out, attention_weights = self.scaled_dot_product_attention(
            WQ_splits, WK_splits, WV_splits, mask)
                        
        out = self.combine_heads(out)
        out = self.linear(out)
            
        return out, attention_weights

In [46]:
#feed forward net
class PoswiseFeedForwardNet(tf.keras.layers.Layer):
    def __init__(self, d_model, d_ff):
        super(PoswiseFeedForwardNet, self).__init__()
        self.d_model = d_model
        self.d_ff = d_ff

        self.fc1 = tf.keras.layers.Dense(d_ff, activation='relu')
        self.fc2 = tf.keras.layers.Dense(d_model)

    def call(self, x):
        out = self.fc1(x)
        out = self.fc2(out)
            
        return out

In [47]:
#encoder layer
class EncoderLayer(tf.keras.layers.Layer):
    def __init__(self, d_model, n_heads, d_ff, dropout):
        super(EncoderLayer, self).__init__()

        self.enc_self_attn = MultiHeadAttention(d_model, n_heads)
        self.ffn = PoswiseFeedForwardNet(d_model, d_ff)

        self.norm_1 = tf.keras.layers.LayerNormalization(epsilon=1e-6)
        self.norm_2 = tf.keras.layers.LayerNormalization(epsilon=1e-6)

        self.do = tf.keras.layers.Dropout(dropout)
        
    def call(self, x, mask):
        '''
        Multi-Head Attention
        '''
        residual = x
        out = self.norm_1(x)
        out, enc_attn = self.enc_self_attn(out, out, out, mask)
        out = self.do(out)
        out += residual
        
        '''
        Position-Wise Feed Forward Network
        '''
        residual = out
        out = self.norm_2(out)
        out = self.ffn(out)
        out = self.do(out)
        out += residual
        
        return out, enc_attn

In [48]:
#decoderlayer
class DecoderLayer(tf.keras.layers.Layer):
    def __init__(self, d_model, num_heads, d_ff, dropout):
        super(DecoderLayer, self).__init__()

        self.dec_self_attn = MultiHeadAttention(d_model, num_heads)
        self.enc_dec_attn = MultiHeadAttention(d_model, num_heads)

        self.ffn = PoswiseFeedForwardNet(d_model, d_ff)

        self.norm_1 = tf.keras.layers.LayerNormalization(epsilon=1e-6)
        self.norm_2 = tf.keras.layers.LayerNormalization(epsilon=1e-6)
        self.norm_3 = tf.keras.layers.LayerNormalization(epsilon=1e-6)

        self.do = tf.keras.layers.Dropout(dropout)
    
    def call(self, x, enc_out, dec_enc_mask, padding_mask):
        '''
        Masked Multi-Head Attention
        '''
        residual = x
        out = self.norm_1(x)
        out, dec_attn = self.dec_self_attn(out, out, out, padding_mask)
        out = self.do(out)
        out += residual

        '''
        Multi-Head Attention
        '''
        residual = out
        out = self.norm_2(out)
        # Q, K, V 순서에 주의하세요!
        out, dec_enc_attn = self.enc_dec_attn(Q=out, K=enc_out, V=enc_out, mask=dec_enc_mask)
        out = self.do(out)
        out += residual
        
        '''
        Position-Wise Feed Forward Network
        '''
        residual = out
        out = self.norm_3(out)
        out = self.ffn(out)
        out = self.do(out)
        out += residual

        return out, dec_attn, dec_enc_attn

In [49]:
#encoder
class Encoder(tf.keras.Model):
    def __init__(self,
                    n_layers,
                    d_model,
                    n_heads,
                    d_ff,
                    dropout):
        super(Encoder, self).__init__()
        self.n_layers = n_layers
        self.enc_layers = [EncoderLayer(d_model, n_heads, d_ff, dropout) 
                        for _ in range(n_layers)]
    
        self.do = tf.keras.layers.Dropout(dropout)
        
    def call(self, x, mask):
        out = x
    
        enc_attns = list()
        for i in range(self.n_layers):
            out, enc_attn = self.enc_layers[i](out, mask)
            enc_attns.append(enc_attn)
        
        return out, enc_attns

In [50]:
#decoder
class Decoder(tf.keras.Model):
    def __init__(self,
                    n_layers,
                    d_model,
                    n_heads,
                    d_ff,
                    dropout):
        super(Decoder, self).__init__()
        self.n_layers = n_layers
        self.dec_layers = [DecoderLayer(d_model, n_heads, d_ff, dropout) 
                            for _ in range(n_layers)]
                            
    def call(self, x, enc_out, dec_enc_mask, padding_mask):
        out = x
    
        dec_attns = list()
        dec_enc_attns = list()
        for i in range(self.n_layers):
            out, dec_attn, dec_enc_attn = \
            self.dec_layers[i](out, enc_out, dec_enc_mask, padding_mask)

            dec_attns.append(dec_attn)
            dec_enc_attns.append(dec_enc_attn)

        return out, dec_attns, dec_enc_attns

In [51]:
class Transformer(tf.keras.Model):
    def __init__(self,
                    n_layers,
                    d_model,
                    n_heads,
                    d_ff,
                    src_vocab_size,
                    tgt_vocab_size,
                    pos_len,
                    dropout=0.2,
                    shared_fc=True,
                    shared_emb=False):
        super(Transformer, self).__init__()
        
        self.d_model = tf.cast(d_model, tf.float32)

        if shared_emb:
            self.enc_emb = self.dec_emb = \
            tf.keras.layers.Embedding(src_vocab_size, d_model)
        else:
            self.enc_emb = tf.keras.layers.Embedding(src_vocab_size, d_model)
            self.dec_emb = tf.keras.layers.Embedding(tgt_vocab_size, d_model)

        self.pos_encoding = positional_encoding(pos_len, d_model)
        self.do = tf.keras.layers.Dropout(dropout)

        self.encoder = Encoder(n_layers, d_model, n_heads, d_ff, dropout)
        self.decoder = Decoder(n_layers, d_model, n_heads, d_ff, dropout)

        self.fc = tf.keras.layers.Dense(tgt_vocab_size)

        self.shared_fc = shared_fc

        if shared_fc:
            self.fc.set_weights(tf.transpose(self.dec_emb.weights))

    def embedding(self, emb, x):
        seq_len = x.shape[1]

        out = emb(x)

        if self.shared_fc: out *= tf.math.sqrt(self.d_model)

        out += self.pos_encoding[np.newaxis, ...][:, :seq_len, :]
        out = self.do(out)

        return out

        
    def call(self, enc_in, dec_in, enc_mask, dec_enc_mask, dec_mask):
        enc_in = self.embedding(self.enc_emb, enc_in)
        dec_in = self.embedding(self.dec_emb, dec_in)

        enc_out, enc_attns = self.encoder(enc_in, enc_mask)
        
        dec_out, dec_attns, dec_enc_attns = \
        self.decoder(dec_in, enc_out, dec_enc_mask, dec_mask)
        
        logits = self.fc(dec_out)
        
        return logits, enc_attns, dec_attns, dec_enc_attns

In [54]:
import numpy as np
VOCAB_SIZE =  20000
# 주어진 하이퍼파라미터로 Transformer 인스턴스 생성
transformer = Transformer(
    n_layers=1,
    d_model=256,
    n_heads=8,
    d_ff=1024,
    src_vocab_size=VOCAB_SIZE,
    tgt_vocab_size=VOCAB_SIZE,
    pos_len=200,
    dropout=0.2,
    shared_fc=True,
    shared_emb=True)
		
d_model = 256

print("슝=3")


# Hyperparameters
# > n_layers: 1
# > d_model: 368
# > n_heads: 8
# > d_ff: 1024
# > dropout: 0.2

# Training Parameters
# > Warmup Steps: 1000
# > Batch Size: 64
# > Epoch At: 10

슝=3


In [60]:
# Learning Rate Scheduler 구현
class LearningRateScheduler(tf.keras.optimizers.schedules.LearningRateSchedule):
    def __init__(self, d_model, warmup_steps=4000):
        super(LearningRateScheduler, self).__init__()
        
        self.d_model = d_model
        self.warmup_steps = warmup_steps
    
    def __call__(self, step):
        arg1 = step ** -0.5
        arg2 = step * (self.warmup_steps ** -1.5)
        
        return (self.d_model ** -0.5) * tf.math.minimum(arg1, arg2)
print("슝=3")

슝=3


In [61]:
learning_rate = LearningRateScheduler(d_model)

optimizer = tf.keras.optimizers.Adam(learning_rate,
                                        beta_1=0.9,
                                        beta_2=0.98, 
                                        epsilon=1e-9)
print("슝=3")

슝=3


In [62]:
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True, reduction='none')

def loss_function(real, pred):
    mask = tf.math.logical_not(tf.math.equal(real, 0))
    loss_ = loss_object(real, pred)

    mask = tf.cast(mask, dtype=loss_.dtype)
    loss_ *= mask

    return tf.reduce_sum(loss_)/tf.reduce_sum(mask)


In [63]:
@tf.function()
def train_step(src, tgt, model, optimizer):
    tgt_in = tgt[:, :-1]  # Decoder의 input
    gold = tgt[:, 1:]     # Decoder의 output과 비교하기 위해 right shift를 통해 생성한 최종 타겟

    enc_mask, dec_enc_mask, dec_mask = generate_masks(src, tgt_in)

    with tf.GradientTape() as tape:
        predictions, enc_attns, dec_attns, dec_enc_attns = \
        model(src, tgt_in, enc_mask, dec_enc_mask, dec_mask)
        loss = loss_function(gold, predictions)

    gradients = tape.gradient(loss, model.trainable_variables)    
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))

    return loss, enc_attns, dec_attns, dec_enc_attns
print("슝=3")

슝=3


In [64]:
# 훈련시키기
EPOCHS = 10

for epoch in range(EPOCHS):
    total_loss = 0
    
    dataset_count = tf.data.experimental.cardinality(train_dataset).numpy()
    tqdm_bar = tqdm(total=dataset_count)
    for step, (enc_batch, dec_batch) in enumerate(train_dataset):
        batch_loss, enc_attns, dec_attns, dec_enc_attns = \
        train_step(enc_batch,
                    dec_batch,
                    transformer,
                    optimizer)

        total_loss += batch_loss
        
        tqdm_bar.set_description_str('Epoch %2d' % (epoch + 1))
        tqdm_bar.set_postfix_str('Loss %.4f' % (total_loss.numpy() / (step + 1)))
        tqdm_bar.update()
     


Epoch  3: 100%|██████████| 505/505 [01:23<00:00,  6.07it/s, Loss 2.8582]

Epoch  2: 100%|██████████| 505/505 [00:32<00:00, 15.41it/s, Loss 1.8196]

Epoch  4: 100%|██████████| 505/505 [00:32<00:00, 15.49it/s, Loss 1.1497]

Epoch  6: 100%|██████████| 505/505 [00:32<00:00, 15.55it/s, Loss 0.9043]

Epoch  8: 100%|██████████| 505/505 [00:32<00:00, 15.56it/s, Loss 0.7207]

Epoch 10: 100%|██████████| 505/505 [00:49<00:00, 15.51it/s, Loss 0.4610]

# NLTK를 활용한 BLEU Score

In [121]:
# 번역 함수
# TensorFlow Tokenizer용 bos_id, eos_id 가져오기
bos_id = tokenizer.word_index['<start>']
eos_id = tokenizer.word_index['<end>']

In [123]:
# TensorFlow Tokenizer용 decode_ids 함수
# 정현님 코드
def decode_ids(tokenizer, sequence):
    return tokenizer.sequences_to_texts([sequence])[0]
    
def translate(tokens, model, src_tokenizer, tgt_tokenizer):
    padded_tokens = tf.keras.preprocessing.sequence.pad_sequences([tokens],
                                                           maxlen=MAX_LEN,
                                                           padding='post')
    ids = []
    output = tf.expand_dims([bos_id], 0)
    for i in range(MAX_LEN):
        enc_padding_mask, combined_mask, dec_padding_mask = \
        generate_masks(padded_tokens, output)

        predictions, _, _, _ = model(padded_tokens, 
                                      output,
                                      enc_padding_mask,
                                      combined_mask,
                                      dec_padding_mask)

        predicted_id = \
        tf.argmax(tf.math.softmax(predictions, axis=-1)[0, -1]).numpy().item()

        if eos_id == predicted_id: 
            result = decode_ids(tgt_tokenizer, ids) 
            return result

        ids.append(predicted_id)
        output = tf.concat([output, tf.expand_dims([predicted_id], 0)], axis=-1)

    result = decode_ids(tgt_tokenizer, ids) 
    return result


In [130]:
#정현님 코드
from nltk.translate.bleu_score import sentence_bleu
from nltk.translate.bleu_score import SmoothingFunction

# BLEU 점수 계산 함수 수정
def eval_bleu_single(model, src_sentence, tgt_sentence, src_tokenizer, tgt_tokenizer, mecab, verbose=True):
    # Mecab을 사용한 토크나이징
    src_tokens_mecab = mecab.morphs(src_sentence)
    tgt_tokens_mecab = mecab.morphs(tgt_sentence)
    
    # 시작 및 종료 토큰 추가
    src_sentence = "<start> " + src_sentence + " <end>"
#     tgt_sentence = "<start> " + tgt_sentence + " <end>"
    tgt_sentence = tgt_sentence


    # TensorFlow Tokenizer를 사용한 숫자 시퀀스 변환
    src_tokens = src_tokenizer.texts_to_sequences([' '.join(src_tokens_mecab)])[0]
    tgt_tokens = tgt_tokenizer.texts_to_sequences([' '.join(tgt_tokens_mecab)])[0]

    if (len(src_tokens) > MAX_LEN): return None
    if (len(tgt_tokens) > MAX_LEN): return None

    reference_mecab = ' '.join(tgt_tokens_mecab)  # Mecab으로 토크나이즈된 참조 문장
    reference = tgt_sentence.split()
    candidate = translate(src_tokens, model, src_tokenizer, tgt_tokenizer).split()

    score = sentence_bleu([reference], candidate,
                          smoothing_function=SmoothingFunction().method1)

    if verbose:
        print("Source Sentence: ", src_sentence)
        print("Model Prediction: ", candidate)
        print("Real (Tokenized): ", reference_mecab)  # 수정된 부분
        print("Real (Original): ", reference)
        print("Score: %lf\n" % score)

    return score

In [132]:
# Q. 인덱스를 바꿔가며 테스트해 보세요
test_idx = 10
mecab = Mecab()

eval_bleu_single(transformer, 
                 questions[5], 
                 answers[5], 
                 tokenizer,
                 tokenizer,
                 mecab)

Source Sentence:  <start> SD카드 망가졌어 <end>
Model Prediction:  ['다시', '새로', '편해요', '.']
Real (Tokenized):  다시 새로 사 는 게 마음 편해요 .
Real (Original):  ['다시', '새로', '사는', '게', '마음', '편해요.']
Score: 0.103052



0.10305224267080783

In [133]:
eval_bleu_single(transformer, 
                 '지루하다, 놀러가고 싶어.', 
                 '잠깐 쉬 어도 돼요 .', 
                 tokenizer,
                 tokenizer,
                 mecab)

Source Sentence:  <start> 지루하다, 놀러가고 싶어. <end>
Model Prediction:  ['좋', '아', '하', '는', '게', '마음', '이', '네요', '.']
Real (Tokenized):  잠깐 쉬 어도 돼요 .
Real (Original):  ['잠깐', '쉬', '어도', '돼요', '.']
Score: 0.023980



0.023980296761827107

In [134]:
eval_bleu_single(transformer, 
                 '간만에 여자친구랑 데이트 하기로 했어.', 
                 '놀이공원 어때요', 
                 tokenizer,
                 tokenizer,
                 mecab)

Source Sentence:  <start> 간만에 여자친구랑 데이트 하기로 했어. <end>
Model Prediction:  ['좋', '아', '하', '는', '집', '에', '타', '는', '거', '같', '아요', '.']
Real (Tokenized):  놀이 공원 어때요
Real (Original):  ['놀이공원', '어때요']
Score: 0.000000



0

In [135]:
eval_bleu_single(transformer, 
                 '오늘 일찍 일어났더니 피곤하다.', 
                 '주무시는 건 어때요', 
                 tokenizer,
                 tokenizer,
                 mecab)

Source Sentence:  <start> 오늘 일찍 일어났더니 피곤하다. <end>
Model Prediction:  ['생각', '이', '많이', '나', '봐요', '.']
Real (Tokenized):  주무시 는 건 어때요
Real (Original):  ['주무시는', '건', '어때요']
Score: 0.000000



0